**Импортируем необходимые библиотеки**

In [1]:
import requests
import xlsxwriter
import pandas as pd
import numpy as np
import sqlite3
from bs4 import BeautifulSoup as bs

Передаем сайту "User-agent":

In [2]:
headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
          'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

Прописываем url с которого начнем парсинг. В поле 'text=' указываем требуемую вакансию. На сайте hh.ru можно настроить дополнительные фильтры поиска, после этого вставить полученную ссылку сюда:

In [3]:
base_url = 'https://hh.ru/search/resume?L_is_autosearch=false&area=113&clusters=true&exp_period=all_time&logic=phrase&no_magic=false&order_by=relevance&pos=position%2Cworkplace_organization%2Cworkplace_position&text=менеджер+по+закупкам&page='

Функция парсинга: получаем список url с резюме, в которых указана желаемая зарплата, затем собираем данные внутри каждого резюме:

In [4]:
def hh_pars(base_url, headers, pages):
    data = {}
    job_title_data = []
    salary_data = []
    gender_data = []
    age_data = []
    address_data = []
    job_experience_data = []
    employers_data = []
    employers_data_summery = []
    key_skills_data = []
    key_skills_data_summery = []
    education_data = []
    education_data_summery = []
    languages_data = []
    languages_data_summery = []
    
    for page in range(0,pages):
        base_url_2 = base_url + str(page) 
        #print(base_url_2)
        urls = []
        session = requests.Session()
        request = session.get(base_url_2, headers=headers)

        #Getting a list of urls with CV
        if request.status_code == 200:
            soup = bs(request.content, 'html.parser')
            divs = soup.find_all('div', attrs = {'itemscope': 'itemscope'})
            for div in divs:
                compensation = div.find('div', attrs= {'class': 'resume-search-item__compensation'}).text
                #Getting CV only with Salory
                if compensation != '':
                    try:
                        href = div.find('a', attrs= {'data-qa': 'resume-serp__resume-title'})['href']
                        href_full =  'https://hh.ru'+ href
                        urls.append(href_full)
                    except:
                        None
            #print('OK')
        else:
            print('ERROR')

        #Parsing inside urls
        for url in urls:
            session_url = requests.Session()
            request_url = session.get(url, headers=headers)
            if request_url.status_code == 200:
                soup_inside_urls = bs(request_url.content, 'html.parser')

                #job title
                job_title_searching = soup_inside_urls.find('h1')
                try:
                    job = job_title_searching.find('span').next.text
                except:
                    job = 'не указано'
                job_title_data.append(job)

                #Desired Salary
                salary_header = soup_inside_urls.find('div', attrs = {'class': 'resume-block-container'})
                salary = salary_header.find('span', attrs = {'data-qa': 'resume-block-salary'}).text
                salary_data.append(salary)

                #Gender
                blok_main_info = soup_inside_urls.find('div', attrs = {'class': 'resume-header'})
                gender = blok_main_info.find('span', attrs = {'itemprop': 'gender'}).text
                gender_data.append(gender)
                try:
                    age = blok_main_info.find('span', attrs = {'data-qa': 'resume-personal-age'}).text
                except:
                    age = np.NaN
                age_data.append(age)

                #Location
                address = blok_main_info.find('span', attrs = {'data-qa': 'resume-personal-address'}).text
                address_data.append(str.upper(address))

                #Experience
                blok_job_experience=soup_inside_urls.find('div', attrs = {'data-qa': 'resume-block-experience'}) 
                try:
                    job_experience = blok_job_experience.find('span', attrs = {'class': 'resume-block__title-text resume-block__title-text_sub'}).text
                except:
                    job_experience = np.NaN
                job_experience_data.append(job_experience)

                #Previous employers
                job_company=soup_inside_urls.find_all('div', attrs = {'itemtype': 'http://schema.org/Organization'})
                for jobs_comp in job_company:
                    obrabotka=jobs_comp.find('div', attrs = {'class': 'resume-block__sub-title'})
                    employers=obrabotka.find('span').text
                    employers_data.append(str.upper(employers))
                employers_data_summery.append(employers_data)
                employers_data = []

                #Key skills
                blok_skills=soup_inside_urls.find_all('span', attrs = {'data-qa': 'bloko-tag__text'}) 
                if blok_skills != []:
                    for key in blok_skills:
                        key_skills=key.find('span').text
                        key_skills_data.append(str.upper(key_skills))
                else:
                    key_skills_data = np.NaN
                key_skills_data_summery.append(key_skills_data)
                key_skills_data = []

                #Education
                blok_education=soup_inside_urls.find_all('div', attrs = {'data-qa': 'resume-block-education-name'}) 
                for edu in blok_education:
                    education=edu.find('span').text
                    education_data.append(str.upper(education))
                    #print(education)
                education_data_summery.append(education_data)
                education_data = []

                #Languages
                blok_languages=soup_inside_urls.find_all('p', attrs = {'data-qa': 'resume-block-language-item'}) 
                for lang in blok_languages:
                    languages=lang.contents[0]
                    try:
                        #languages_in_ru = city_translator(languages)
                        languages_data.append(str.upper(languages))
                    except:
                        languages_data = np.nan
                    #print(languages)
                languages_data_summery.append(languages_data)
                languages_data = []

                #Getting final dataset in pandas format
                data = {'Желаемая вакансия': job_title_data,
                                         'Желаемая зарплата': salary_data, 
                                         'Пол': gender_data, 
                                         'Возраст': age_data, 
                                         'Местонахождения': address_data, 
                                         'Общий стаж': job_experience_data, 
                                         'Работодатели':  employers_data_summery,
                                         'Ключевые навыки': key_skills_data_summery,
                                         'Образование': education_data_summery, 
                                         'Знание языков': languages_data_summery}    

                #Inputing list in Pandas cells
                data_pd = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()]))
               
            else:
                print('Error')
        print('.', end='')
    
    print('\n','Parsing was completed')
    return data_pd

Вызываем функцию парсинга: передаем ссылку, данные "user - agent", общее количество страниц для парсинга:

In [5]:
data_table=hh_pars(base_url, headers, 10)

..........
 Parsing was completed


Записываем данные в excel

In [6]:
writer = pd.ExcelWriter('parsing_final_dataset.xlsx', engine='xlsxwriter')
data_table.to_excel(writer, 'Sheet1')
writer.save()

In [7]:
data_table

,Желаемая вакансия,Желаемая зарплата,Пол,Возраст,Местонахождения,Общий стаж,Работодатели,Ключевые навыки,Образование,Знание языков
0,Менеджер по закупкам,35 000 руб.,Женщина,36 лет,ЛИПЕЦК,Опыт работы 6 лет 4 месяца,"[ООО ОРИОН, ООО НИКА]","[ЗАКУПКИ, РАБОТА С ПОСТАВЩИКАМИ, ЗАКУПКА ТОВАР...",[],[РУССКИЙ]
1,Менеджер по закупкам,40 000 руб.,Женщина,NaN,КАЛУГА,Опыт работы 9 лет,"[ИП ЛУЦЕНКО, ООО ДИМ, ООО СПОРТ ПЛЮС]","[ABC-АНАЛИЗ, XYZ-АНАЛИЗ, ОБОРАЧИВАЕМОСТЬ ТОВАР...",[СЕВЕРО-ЗАПАДНАЯ АКАДЕМИЯ ГОСУДАРСТВЕННОЙ СЛУЖБЫ],"[РУССКИЙ, АНГЛИЙСКИЙ]"
2,Менеджер по закупкам,60 000 руб.,Мужчина,46 лет,САНКТ-ПЕТЕРБУРГ,Опыт работы 9 лет 7 месяцев,"[КРОНДЕ ГРУПП, ООО ""ЭЛКОМ"", ООО ""ВЕНТПРОФСИСТЕ...","[ЗАКУПКИ, ПОИСК ИНФОРМАЦИИ В ИНТЕРНЕТ, ГРАМОТН...",[БАЛТИЙСКИЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИВЕР...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
3,Менеджер по закупкам..,70 000 руб.,Женщина,41 год,МОСКВА,Опыт работы 11 лет 6 месяцев,"[ПКК КАЛИБР, ООО ""ТОРГОВАЯ ДИСТРИБЬЮТОРСКАЯ КО...","[НАВЫКИ МЕЖЛИЧНОСТНОГО ДЕЛОВОГО ОБЩЕНИЯ., СПОС...",[ДАГЕСТАНСКИЙ ГОСУДАРСТВЕННЫЙ ПЕДАГОГИЧЕСКИЙ У...,[РУССКИЙ]
4,Менеджер по закупкам,40 000 руб.,Женщина,29 лет,НИЖНИЙ НОВГОРОД,Опыт работы 9 лет 3 месяца,"[ООО ИНСТРУМАВИА, ООО ИНСТРУМЕНТ, НИЖЕГОРОДСКИ...","[ЗАКЛЮЧЕНИЕ ДОГОВОРОВ, ДОГОВОРНАЯ РАБОТА, ДОКУ...",[НИЖЕГОРОДСКИЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИ...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
...,...,...,...,...,...,...,...,...,...,...
149,Менеджер по закупкам / Менеджер по работе с кл...,40 000 руб.,Женщина,32 года,РОСТОВ-НА-ДОНУ,Опыт работы 11 лет 11 месяцев,"[ПОБЕДА, ООО, ПОБЕДА, ООО, ООО ""ФИРМА ВИСТ-РОС...","[1С: ТОРГОВЛЯ И СКЛАД, 1С: ТОРГОВЛЯ, ДОКУМЕНТА...",[ЗЕРНОГРАДСКИЙ ПЕДЕГОГИЧЕСКИЙ КОЛЛЕДЖ],"[РУССКИЙ, АНГЛИЙСКИЙ]"
150,"Специалист по складской логистике, Менеджер по...",35 000 руб.,Мужчина,34 года,ВОЛЖСКИЙ(ВОЛГОГРАДСКАЯ ОБЛАСТЬ),Опыт работы 11 лет 2 месяца,"[ГАНИМЕД СБ, ООО, ВЫМПЕЛ- КОММУНИКАЦИИ, ПАО (Б...",NaN,"[ВОЛГОГРАДСКИЙ ГОСУДАРСТВЕННЫЙ УНИВЕРСИТЕТ, ЭФ...","[РУССКИЙ, АНГЛИЙСКИЙ, НЕМЕЦКИЙ]"
151,Менеджер по закупкам,30 000 руб.,Мужчина,47 лет,ЭНГЕЛЬС,Опыт работы 25 лет 10 месяцев,"[КОМЕРЧЕСКАЯ ФИРМА, ОАО ""ЗАЗС"" ГРУППА БОШ, ОАО...","[ЗАКУПКИ, ЛОГИСТИКА ПОСТАВОК, ЭЛЕКТРОННАЯ ПОЧТ...",[ЭНГЕЛЬССКИЙ ТЕХНОЛОГИЧЕСКИЙ ИНСТИТУТ САРАТОВС...,[РУССКИЙ]
152,Менеджер по закупкам,80 000 руб.,Мужчина,42 года,МОСКВА,Опыт работы 18 лет,"[ООО ""ПАЛЛЕТТРЕЙД"", ООО ""СМАРТ ГЕЙМЗ"", ООО ""МО...","[БЫСТРАЯ ОБУЧАЕМОСТЬ, САМОСТОЯТЕЛЬНОСТЬ В ПРИН...",[МОСКОВСКИЙ АВИАЦИОННЫЙ ИНСТИТУТ (НАЦИОНАЛЬНЫЙ...,[АНГЛИЙСКИЙ]


Опционально записываем данные в базу данных SQLite

In [10]:
import json

#convert lists to JSON format function:

def adapt_list_to_JSON(lst):
    return json.dumps(lst).encode('utf8')

def convert_JSON_to_list(data):
    return json.loads(data.decode('utf8'))

sqlite3.register_adapter(list, adapt_list_to_JSON)
sqlite3.register_converter("json", convert_JSON_to_list)

#Write pandas dataset to SQLite database

conn = sqlite3.connect('parsing_final_dataset.db', detect_types=sqlite3.PARSE_COLNAMES)
curs = conn.cursor
data_table.to_sql('parsing_final_dataset', conn, if_exists='replace', index=False)

Извлекаем необходимые столбцы из SQL c автоматической конвертацией из JSON обратно в списки

In [11]:
pd.read_sql('SELECT "Желаемая вакансия", "Желаемая зарплата", "Пол", "Возраст", "Общий стаж", "Работодатели" AS "Работодатели [json]", "Ключевые навыки" AS "Ключевые навыки[json]", "Образование" AS "Образование[json]", "Знание языков" AS "Знание языков[json]" FROM parsing_final_dataset', conn)

,Желаемая вакансия,Желаемая зарплата,Пол,Возраст,Общий стаж,Работодатели,Ключевые навыки,Образование,Знание языков
0,Менеджер по закупкам,35 000 руб.,Женщина,36 лет,Опыт работы 6 лет 4 месяца,"[ООО ОРИОН, ООО НИКА]","[ЗАКУПКИ, РАБОТА С ПОСТАВЩИКАМИ, ЗАКУПКА ТОВАР...",[],[РУССКИЙ]
1,Менеджер по закупкам,40 000 руб.,Женщина,None,Опыт работы 9 лет,"[ИП ЛУЦЕНКО, ООО ДИМ, ООО СПОРТ ПЛЮС]","[ABC-АНАЛИЗ, XYZ-АНАЛИЗ, ОБОРАЧИВАЕМОСТЬ ТОВАР...",[СЕВЕРО-ЗАПАДНАЯ АКАДЕМИЯ ГОСУДАРСТВЕННОЙ СЛУЖБЫ],"[РУССКИЙ, АНГЛИЙСКИЙ]"
2,Менеджер по закупкам,60 000 руб.,Мужчина,46 лет,Опыт работы 9 лет 7 месяцев,"[КРОНДЕ ГРУПП, ООО ""ЭЛКОМ"", ООО ""ВЕНТПРОФСИСТЕ...","[ЗАКУПКИ, ПОИСК ИНФОРМАЦИИ В ИНТЕРНЕТ, ГРАМОТН...",[БАЛТИЙСКИЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИВЕР...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
3,Менеджер по закупкам..,70 000 руб.,Женщина,41 год,Опыт работы 11 лет 6 месяцев,"[ПКК КАЛИБР, ООО ""ТОРГОВАЯ ДИСТРИБЬЮТОРСКАЯ КО...","[НАВЫКИ МЕЖЛИЧНОСТНОГО ДЕЛОВОГО ОБЩЕНИЯ., СПОС...",[ДАГЕСТАНСКИЙ ГОСУДАРСТВЕННЫЙ ПЕДАГОГИЧЕСКИЙ У...,[РУССКИЙ]
4,Менеджер по закупкам,40 000 руб.,Женщина,29 лет,Опыт работы 9 лет 3 месяца,"[ООО ИНСТРУМАВИА, ООО ИНСТРУМЕНТ, НИЖЕГОРОДСКИ...","[ЗАКЛЮЧЕНИЕ ДОГОВОРОВ, ДОГОВОРНАЯ РАБОТА, ДОКУ...",[НИЖЕГОРОДСКИЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИ...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
...,...,...,...,...,...,...,...,...,...
149,Менеджер по закупкам / Менеджер по работе с кл...,40 000 руб.,Женщина,32 года,Опыт работы 11 лет 11 месяцев,"[ПОБЕДА, ООО, ПОБЕДА, ООО, ООО ""ФИРМА ВИСТ-РОС...","[1С: ТОРГОВЛЯ И СКЛАД, 1С: ТОРГОВЛЯ, ДОКУМЕНТА...",[ЗЕРНОГРАДСКИЙ ПЕДЕГОГИЧЕСКИЙ КОЛЛЕДЖ],"[РУССКИЙ, АНГЛИЙСКИЙ]"
150,"Специалист по складской логистике, Менеджер по...",35 000 руб.,Мужчина,34 года,Опыт работы 11 лет 2 месяца,"[ГАНИМЕД СБ, ООО, ВЫМПЕЛ- КОММУНИКАЦИИ, ПАО (Б...",None,"[ВОЛГОГРАДСКИЙ ГОСУДАРСТВЕННЫЙ УНИВЕРСИТЕТ, ЭФ...","[РУССКИЙ, АНГЛИЙСКИЙ, НЕМЕЦКИЙ]"
151,Менеджер по закупкам,30 000 руб.,Мужчина,47 лет,Опыт работы 25 лет 10 месяцев,"[КОМЕРЧЕСКАЯ ФИРМА, ОАО ""ЗАЗС"" ГРУППА БОШ, ОАО...","[ЗАКУПКИ, ЛОГИСТИКА ПОСТАВОК, ЭЛЕКТРОННАЯ ПОЧТ...",[ЭНГЕЛЬССКИЙ ТЕХНОЛОГИЧЕСКИЙ ИНСТИТУТ САРАТОВС...,[РУССКИЙ]
152,Менеджер по закупкам,80 000 руб.,Мужчина,42 года,Опыт работы 18 лет,"[ООО ""ПАЛЛЕТТРЕЙД"", ООО ""СМАРТ ГЕЙМЗ"", ООО ""МО...","[БЫСТРАЯ ОБУЧАЕМОСТЬ, САМОСТОЯТЕЛЬНОСТЬ В ПРИН...",[МОСКОВСКИЙ АВИАЦИОННЫЙ ИНСТИТУТ (НАЦИОНАЛЬНЫЙ...,[АНГЛИЙСКИЙ]
